### Actividad 2
Realizar un análisis de vulnerabilidad con Nmap en el servidor `ahg.uv.cl`, enfocado en los puertos 22, 80, 443, 444, 445 y 53. Interpretaré los resultados para ofrecer recomendaciones de ciberseguridad y desarrollaré reglas de firewall adecuadas. Además, propondré cuatro medidas de mitigación para reducir riesgos.

In [9]:
import subprocess
import os
from datetime import datetime

# Definir las variables para el comando nmap
ip_address = "ahg.uv.cl" # IP de la máquina a escanear
scan_types = ["-sS", "-sA", "-sX", "-Pn", "-sN"]
# -sS para detectar los puertos abiertos (EScaneo SYN)
# -sA para detectar los puertos filtrados (es decir puertos cerrados con un firewall) (Escaneo ACK)
# -sX ataque de XMAS scan (engañar al firewall, para obtener datos de los puertos cerrados) 
# -Pn para realizar escaneos sin ping (donde se tiene un firewall que bloquea los pings)
# -sN para confirmar si el host está cerrado (responde con RST)
scan_service = "-sV" # -sV para detectar los servicios
scan_os = "-O" # -O para detectar el sistema operativo
port_range = "-p 22,80,443,444,445,53" # -p para especificar el rango de puertos a escanear
# Puerto 22: Utilizado para conexiones seguras SSH para administración remota.
# Puerto 80: Estándar para tráfico web HTTP no cifrado.
# Puerto 443: Usado para tráfico web HTTPS cifrado.
# Puerto 444: Comúnmente utilizado para servicios personalizados; no asignado oficialmente.
# Puerto 445: Empleado para compartir archivos y impresoras en redes Windows (SMB).
# Puerto 53: Maneja consultas DNS para la resolución de nombres de dominio a direcciones IP.
# output_format = "-oX tmp/nmap_output3.xml --append-output" 

# Crear la carpeta para almacenar los resultados si no existe
folder_name = "nmap_results"
os.makedirs(folder_name, exist_ok=True)

# Función para formatear el nombre del archivo basado en el tipo de escaneo y la fecha actual
def get_output_filename(scan_type):
    date_str = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    return os.path.join(folder_name, f"nmap_output_{scan_type.strip('-')}_{date_str}.xml")

# Ejecutar los escaneos y guardar cada resultado en un archivo separado
for scan_type in scan_types:
    output_file = get_output_filename(scan_type)
    output_format = f"-oX {output_file}"
    
    
    # Formatear el comando usando f-strings
    command = f"nmap {scan_type} {ip_address} {scan_service} {scan_os} {output_format} {port_range}"

    # Ejecutar el comando nmap
    result = subprocess.run(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

    # Imprimir la salida y los errores
    print("Salida:", result.stdout)
    if result.stderr:
        print("Error:", result.stderr)

Salida: Starting Nmap 7.95 ( https://nmap.org ) at 2024-05-07 16:10 Hora est. Sudamérica Pacífico
Nmap scan report for ahg.uv.cl (200.14.249.53)
Host is up (0.022s latency).

PORT    STATE    SERVICE      VERSION
22/tcp  filtered ssh
53/tcp  filtered domain
80/tcp  open     http         Apache httpd
443/tcp open     ssl/http     Apache httpd
444/tcp filtered snpp
445/tcp filtered microsoft-ds
Device type: general purpose|router
Running (JUST GUESSING): Linux 4.X|5.X|2.6.X|3.X (92%), MikroTik RouterOS 7.X (85%)
OS CPE: cpe:/o:linux:linux_kernel:4 cpe:/o:linux:linux_kernel:5 cpe:/o:mikrotik:routeros:7 cpe:/o:linux:linux_kernel:5.6.3 cpe:/o:linux:linux_kernel:2.6.32 cpe:/o:linux:linux_kernel:3.10
Aggressive OS guesses: Linux 4.19 - 5.15 (92%), Linux 4.15 - 5.19 (85%), Linux 4.19 (85%), Linux 5.0 - 5.14 (85%), OpenWrt 21.02 (Linux 5.4) (85%), MikroTik RouterOS 7.2 - 7.5 (Linux 5.6.3) (85%), Linux 2.6.32 (85%), Linux 2.6.32 or 3.10 (85%), Linux 4.0 - 4.4 (85%), Linux 4.15 (85%)
No exact OS 

In [1]:
import subprocess
import os
import xml.etree.ElementTree as ET
import pandas as pd

def run_nmap_scan(ip, ports, scan_type):
    output_xml = f"nmap_results/nmap_output_{ip}_{scan_type.strip('-')}.xml"
    output_txt = f"nmap_results/nmap_output_{ip}_{scan_type.strip('-')}.txt"
    command = f"nmap {scan_type} -sV -O -oX {output_xml} -oN {output_txt} {ports} {ip}"
    subprocess.run(command, shell=True)
    return output_xml, output_txt

def parse_nmap_xml(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    scan_data = []
    for host in root.findall('host'):
        for port in host.find('ports').findall('port'):
            port_id = port.get('portid')
            service = port.find('service').get('name', 'unknown')
            product = port.find('service').get('product', 'unknown')
            version = port.find('service').get('version', 'unknown')
            state = port.find('state').get('state')
            scan_data.append((port_id, service, product, version, state))
    return pd.DataFrame(scan_data, columns=['Port', 'Service', 'Product', 'Version', 'State'])

def analyze_vulnerabilities(df, ip):
    print(f"\nAnalysis for {ip}:")
    open_ports = df[df['State'] == 'open']
    filtered_ports = df[df['State'] == 'filtered']
    closed_ports = df[df['State'] == 'closed']
    
    if not open_ports.empty:
        print("Open Ports:")
        print(open_ports[['Port', 'Service', 'Product', 'Version']])
    
    if not filtered_ports.empty:
        print("Filtered Ports:")
        print(filtered_ports[['Port', 'Service']])
        
    if not closed_ports.empty:
        print("Closed Ports:")
        print(closed_ports[['Port', 'Service']])
    
    # Suggest firewall rules
    if not open_ports.empty:
        print("Suggested Firewall Rules:")
        for index, row in open_ports.iterrows():
            if 'http' in row['Service']:
                print(f"Allow {row['Port']} for HTTP/HTTPS traffic")
            else:
                print(f"Block {row['Port']} to reduce attack surface")

if __name__ == "__main__":
    if not os.path.exists('nmap_results'):
        os.makedirs('nmap_results')

    servers = {
        "ahg.uv.cl": "-p 22,80,443,444,445,53",
        "itmon-uni.uv.cl": "-p 22,80,443,53,135,137,139",
        "ingenieriaoceanica.uv.cl": "" 
    }
    
    scan_types = ["-sS", "-sA", "-sX", "-Pn", "-sN"]

    all_results = {}

    for ip, ports in servers.items():
        combined_df = pd.DataFrame()
        for scan_type in scan_types:
            xml_output, txt_output = run_nmap_scan(ip, ports, scan_type)
            df = parse_nmap_xml(xml_output)
            combined_df = pd.concat([combined_df, df]).drop_duplicates().reset_index(drop=True)

        all_results[ip] = combined_df
    
    # Analyze all results after all scans
    for ip, combined_df in all_results.items():
        analyze_vulnerabilities(combined_df, ip)


In [1]:
import subprocess
import os
import xml.etree.ElementTree as ET
import pandas as pd

def run_nmap_scan(ip, ports, scan_type):
    output_xml = f"nmap_results/nmap_output_{ip}_{scan_type.strip('-')}.xml"
    output_txt = f"nmap_results/nmap_output_{ip}_{scan_type.strip('-')}.txt"
    command = f"nmap {scan_type} -sV -O -oX {output_xml} -oN {output_txt} {ports} {ip}"
    subprocess.run(command, shell=True)
    return output_xml, output_txt

def parse_nmap_xml(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    scan_data = []
    for host in root.findall('host'):
        for port in host.find('ports').findall('port'):
            port_id = port.get('portid')
            service = port.find('service').get('name', 'unknown')
            product = port.find('service').get('product', 'unknown')
            version = port.find('service').get('version', 'unknown')
            state = port.find('state').get('state')
            scan_data.append((port_id, service, product, version, state))
    return pd.DataFrame(scan_data, columns=['Port', 'Service', 'Product', 'Version', 'State'])

def analyze_vulnerabilities(df, ip):
    print(f"\nAnalysis for {ip}:")
    open_ports = df[df['State'] == 'open']
    filtered_ports = df[df['State'] == 'filtered']
    closed_ports = df[df['State'] == 'closed']
    
    if not open_ports.empty:
        print("Open Ports:")
        print(open_ports[['Port', 'Service', 'Product', 'Version']])
    
    if not filtered_ports.empty:
        print("Filtered Ports:")
        print(filtered_ports[['Port', 'Service']])
        
    if not closed_ports.empty:
        print("Closed Ports:")
        print(closed_ports[['Port', 'Service']])
    
    # Suggest firewall rules
    if not open_ports.empty:
        print("Suggested Firewall Rules:")
        for index, row in open_ports.iterrows():
            if 'http' in row['Service']:
                print(f"Allow {row['Port']} for HTTP/HTTPS traffic")
            else:
                print(f"Block {row['Port']} to reduce attack surface")

if __name__ == "__main__":
    if not os.path.exists('nmap_results'):
        os.makedirs('nmap_results')

    servers = {
        "ahg.uv.cl": "-p 22,80,443,444,445,53",
        "itmon-uni.uv.cl": "-p 22,80,443,53,135,137,139",
        "ingenieriaoceanica.uv.cl": ""  # Escaneo general modificado
    }
    
    scan_types = ["-sS", "-sA", "-sX", "-Pn", "-sN"]

    all_results = {}

    for ip, ports in servers.items():
        combined_df = pd.DataFrame()
        for scan_type in scan_types:
            xml_output, txt_output = run_nmap_scan(ip, ports, scan_type)
            df = parse_nmap_xml(xml_output)
            combined_df = pd.concat([combined_df, df]).drop_duplicates().reset_index(drop=True)

        all_results[ip] = combined_df
    
    # Analyze all results after all scans
    for ip, combined_df in all_results.items():
        analyze_vulnerabilities(combined_df, ip)
